In [1]:
# handy imports to have
%matplotlib inline
from sympy import symbols, Eq, solve
import math
from itertools import combinations_with_replacement , permutations, combinations, count, chain
import numpy as np
import time
import functools
from tqdm.notebook import trange, tqdm
import ipywidgets as widgets
import matplotlib.pyplot as plt
from collections import Counter, deque
import more_itertools as mit
import string
import ast
import networkx as nx

In [147]:
shape_names_dict = {0: 'square', 1:'Horiz Bar', 2:'Plus', 3:'Backwards L', 4:'Vert Bar'}

class block:
    
    def __init__(self, sequence_num, highest):
        self.shape = sequence_num % 5
        self.shape_name = shape_names_dict[self.shape]
        
        match self.shape:
            
            case 0:   # square
                self.coords = [(highest+4, i) for i in range(3, 5)] + [(highest+5,i) for i in range(3,5)]
            case 1:  # horiz bar
                self.coords = [(highest+4, i) for i in range(3, 7)]
            case 2:  # Plus
                self.coords = [(highest+5, i) for i in range(3, 6)] + [(highest+4, 4), (highest+6, 4)]
            case 3:  # backwards L
                self.coords = [(highest+4, i) for i in range(3, 6)] + [(i, 5) for i in range(highest+5, highest+7)]
            case 4: 
                self.coords = [(i, 3) for i in range(highest+4, highest+8)]
            case _:
                assert False, 'got an invlalid shape number/name'
    
    def get_coords(self):
        return self.coords
    
    # could probably refactor these three into one call with differnt ifs, but is OK as is.
    def can_move_left(self, c):
        for y,x in self.coords:
            if c[y][x-1] == '|' or c[y][x-1] == '#':
                return False
        return True
    
    def can_move_right(self, c):
        for y,x in self.coords:
            if c[y][x+1] == '|' or c[y][x+1] == '#':
                return False
        return True
    
    def is_placed(self, c):
        for y,x in self.coords:
            if c[y-1][x] == '|' or c[y-1][x] == '#' or c[y-1][x] == '-':
                return True
        return False
        
    
    def move(self, c, direction):
        if direction == '<':   # if blowing left
            if self.can_move_left(c):
                self.coords = [(y,x-1) for y,x in self.coords]
                return True
        elif direction == '>':
            if self.can_move_right(c):
                self.coords = [(y,x+1) for y,x in self.coords]
                return True
        elif direction == 'down':
            if not self.is_placed(c):
                self.coords = [(y-1,x) for y,x in self.coords]
                return True
        
        return False
            
        
        

In [135]:
# sample input
wind_input = '>>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>'

In [160]:
# puzzle input
wind_input = '>>><<<>><<<>>><<<>>><<<>><<><>>><<>>><>><<<>>><<><>>><<<<>>>><>><<>>>><>>>><<<<>>>><<<>>><>><<>><<<>><<<<><<>><<><<<>><<>>><<<>>>><>>><>><><<<<><>><><<>>>><<><<<<><<<><<>>>><<><<>><<<<>>>><>>>><>><><<<>>>><>>>><>><<>>>><>>>><<><<<<>><<>>>><>>><>><>><<>><<<<>>>><<<>><<<><><<<<>><<><<<<><<<><<<<>>>><<><<<>>><<>><>><<><<<<>>>><<<>>>><<<<>><<<<>><<>>>><<><>><>>>><<<<>>>><<>>><<<><<<<>>><>><<<>>><<<>>>><<<>>>><>>>><>>><<<<>>>><<<>>><<<<>>><><<>>><<<>>>><<<>>>><>>>><<<<>><<>>><<<<><>>>><><<<<>>><<<<>>>><<<>>>><<>>><>><<>><<>><<>>><<>>>><>>><<<><<<>><<<<>>><<<>>><<<><>>><<<<><>><<<>>>><><<<<><<>><<<<>><<<>>><>><<><><<<>><<<>>><<>>>><<>><><<>>>><<<<>>><<<>><<<<>><<><<<>>>><>>><>><<<>>>><<<>>><>><<<<><><<>><<<>><<<<>>>><<<<>>><<<<>><>>>><>>><>>><<>>><<<<>><<><<<<>>>><<<<>>>><<<><<>><><<<>>><>><<>><<<>>><>>><<<<>><>><><<><>><>>><>>>><<<>>><<<<>>>><<<>>>><<<>><<<<>>>><<<>>><><<>>><>>><<<>>><<<<><>>>><<<><>>>><>>><<>>><>><<<<>>><<>><<<<>><<>>>><<<><<><><<<>><<<<>>><<<><<<<>>>><<<<>>>><<<<>><<>>><<<><<<<><<<>>><><<<<><>><<><<<<>>><<>><>><<<><<<>>><<>><<<<><<<>><<<<>>><<>>><<<>>>><<<>><<<>>>><><<>><>>><<<>><><>><<>>><<<<>>><<>>>><>><<><<<>>>><<<>>><<>><<<<>>><<<<><<<<><<>>>><>><><<<>>>><<<<>><<<<>><>><<>><<<>><><<<<>>>><<<<>>>><<<>>>><>><<>><<<><<<<>><<<<><<<><<<>>><<<>><>>><<>><<><<>><<>><<><<<><<<<>>>><<<><>>>><<>><<>>><>>><<>>><><<>>><<<>>>><<<<>>>><><<<>>><<>>>><<<<><>>><>><<>><<<><><>>><<<>>>><>>><<><>>><<>>><<<>>><<<><<<>>><<<<><<<<>>>><<<>>>><<<<>>>><<<>>><<>><<>>><<<<>><><<<><<><<<>>>><<<<>>><>><<<<><<<>>>><<<<>>>><<<>><>>>><<<><<<<>>>><>>>><<>>>><<<><<<<><<>><>>>><<<>>>><<<<>>><>>>><<<><<>>><<>><<<>><>><<<>>><<>>>><<>>><<>>>><<><<>>>><<<<>>><<<<>>><<<>>><>><<><<<<>>>><<<>>><><<<<>>><<>>><<>><<<>><<<<><><<<<>>>><<<><<>>>><<<>>>><>><<<>>>><>>>><><<<<>><<<>>><>>><>>><<<>><<<<><>>><<>>><<<>>>><>>>><><<<<>>>><<>>><>>><<<<><<>>><<<><<<>>>><<<<><><<<<><<<<>><<<><>><<<<>>><>><<><<>>><<<>>><<<<>><<<>>><<<>>>><<<<>>><>><>><<<<>>>><>>><<<<>>><<>>><>><<<><>>>><<<<><<<>><><<<<>><<<<>>>><>><>><>>>><<<>>>><<>>>><<<>>><<<<>>>><<<<>>>><<><><<<>>><<>>><<<<>>>><<<<><<>>>><<<><<<<>>>><<>><<<>><<><>><<<<>>><<<<>>><<<<>>><<>>>><<<<>><<>>><<>>><>>>><<<>>>><<>>><<<<><<<>>><<<<><<><<>>><<>>>><<<<>>><<>>><<<<>>>><<<<>>><<<<><>><<<>>>><<<<>><<<>>><<<><<>>>><>>><<>>><>>>><<<><<<<><<<><<<>>>><<>>>><>>><>>>><<>>>><<<<>>>><>><<<<><<<>>>><><<>>>><<><<<>><<<<>>>><<<<><<<><<<<>>>><>><><<<<>><<<<>>><<>><<>>><>>><<<>>>><<<<>>><<<<>>><<>>>><<>><>><<<<>>>><<<>><<>><><><<>><<<<>>><<><<<<><><<<>>>><<<<>>><<<<>>>><<<>>><<<<><<<<>>><<><<><<>>><<<><<>><>>>><<<>><<>>><<<<>>>><>>>><<<>>>><<<<>><<><<<>>>><<<>>><>>>><<<<>>>><<<<><<>>><<<<>>>><<>>>><<<<><<<<>>><<>>><><<<<><<>>><>>>><<><<<>>>><>>><<<>><<<>><<<<>><>>>><<<>><<<<><>>>><<<>><<<<>>>><<<<>><<<<>><<>>><>>>><<>>><<<<><<>><>><<<<>>>><<<<>>>><<>><<<<><<<<><<>><><<>>>><<<<>>><<<<>><<<<>><<<<>><<<<>>><<<<>>><<<<><<<>>><><>>>><<>><>>><<>>><<<>>><<><>>>><<<<>>><<<>><<<<>>><<<>>><<>><<>><><<>><<>><<>>>><>>><<<>>>><<<<>>>><<<>><>>>><<<>>><<<><<>>>><<<>><<<<>><<<>>><<<<>><<><<<>>>><<>><>>>><<<<><>>>><<><<<<>><<>>><<>><<<<>>>><<>>><<>><<>>><>>>><<<>><<<><>><<<<>>><<<>>><<><<>>>><<<><>>>><>><<<>><><<<<>><<<<>>>><<<<>><><<<>>>><><<<<>>>><<>>><>>>><<>><>><<<><>><<<>>>><>><<<><<><><<<>>><<>><<<<>><<<>>><<>><<><>><<<<>>>><<<<>><<>>>><<<>><<><>>>><<<<>>>><>>><<>>><><<<<>><<>><>>>><<<<>>>><><<>>>><>>>><<<<>>>><<<<><>><><<<>>>><>>><<><><<<><<<<>>>><<<<>><<<>>>><<<<>>>><<<<>><<>>>><>><>>>><<<<><<>><<<><<<>>>><<<>>><<>>>><>><<<<>><<<>>>><<>>>><<<<>>>><<<<><<<>><>><><><<<><<<><<<><<<<>>><<>>><<>>><<<>><<<><<<<><<>>>><>><>><<<>>><<<<>>>><<<<>>><<<<>>><<<>>>><<>><<>><>><<>>><<>>>><<<<>>>><>>>><><><<<<><<<<>><<<><<<>>><<>>><<<>>>><<>>>><<>>><<<<>>>><<<><<<<>>><<<<><<>>>><<<<>>><>><><<>>><>>>><>>><<<>><<<<>><<<>>>><<>><<<>>>><<<><<<<>>><<<<><<<><<<>><<<>>>><<>>><>><<<<>>><<<>><>>>><<><><<<><><<<>>>><<<>><<><<<>>>><<<>>>><>>>><<<>>><<<>>><<<<>><<>><>><<>><>>><<><>><>>>><<<><<<><<<><<<<>><<>>>><>>>><>>><<<<><<<<>>>><<><<<>>><<<>>>><<<>>><<>>>><<>><><<<>><<<<>>>><<<<><<>><>><<<>><><<>><<<<>>><>>>><<<>><<<>><<<>>><<><<<><>><><><<>><<<>>><<<><<<><>>>><>>>><>>><<<<>>>><><<<<>>><<>><<<<>><<>>><>>>><<>>><>>>><<<>>>><<<>>>><<<<>>><<<<>><<<>>><<<>><<<>>><<<<>>><<>>>><<>>>><<<<>>>><<<<>><<<>>><<<<>>><>>>><<<<><<<<>><<<<>>>><>>><<<>><<>>>><<><<>><<<>>><<<<><<<<>>>><<<<><<<>>>><<<<>>><><<<<><<<<>><><<<>>>><>><<<<>><><>><<<>>>><><<>>>><><<<>>>><>>><<<<>>>><<<<>>><<<<>><<>>>><>>>><>>><<<><<>>>><<<<><<<<>><><>><<<>><<><<<<>>><<<<>>>><<<>><>><<>>><<<>>><<<<>>><<<><>>><<<>>><<<>><<<>>>><<<>><<>>>><<<<>>><<<<>><<<><>><<<>><><<<>><<<<><><>><<<><>><><<<>>><>>>><>>>><<<>><<<>>>><<<>>><<<<>>><<<>>>><<>>>><>><<><<<>><<>>>><>>>><<>><<<<><<<>><<>><<<><<>>>><<<<><<<>>>><<><<>>>><<><>>><><<>>><<<><<><<<<>>><<>>><>>>><><>>><<<<>><<>><<>>>><<<<>>><<<>>><<><<<><<<<>>><<>>><>><>>>><<><<<>>><<<<>>>><<>><<<<>><>>><<<>>><<>>>><<<>><<><<<>>><><<>>><<><<>>><>>>><<<<>>>><<<<>>>><<<>>>><<<<>>>><><<<<>>><>>><<<<>>><<>><<<>>>><>>>><><<<>><<>><<<>>><<<<>>><<<>>>><><<<<>>>><<><<>>><<<<>>>><<>>><<<<><>>><<<>>><<><<><<<>><>>>><<<>>>><<<>><<<>><<>>><<>>>><<<><>>>><<<<>>><<<<>><<>><<<>>><<<<>><<<<>>>><<>><<>><<<<>>>><>>>><<<>><>>><><>>><>>>><<<>>>><<>><<<>><<>><>>>><<<>>>><<<<><>>><<><<><>>>><<>>>><>>><<<<>>>><>>>><<<><<<><>>><<<>>><>>><<<>><<<<>>><>>>><<><<<<><<<>>>><<<<><<<<>><<>>><<<<><<<<>>><<>>><<<>>>><<>>>><<<>>>><<>><<>>><<<>>><<>>><>>><<>><<>><<<<><<>>>><<<>>>><<<>><<<<>>>><<>>>><>>>><>><>>>><><<<>>>><<<>>><><<<<>>><<<>><<<>><<<<>><<<>>>><<<<>>>><>>>><>><<<><><>>><>>>><<<<>>>><<<>>>><<>><>>>><<<<><<<>>>><>><<<>><<<<>>>><>>><<<<>>><<<<>><<<>>><<<<>>>><<>>><<>><<<>>><>><<>>>><<<<>>><<<<>>><<<<><<<<>>>><<<<><<<<>><>>><><<<>>><<<<>><<<<>>>><<<>>><>>><>>><<<>>><>>>><>><<>><<<<><<><>>>><<<<>>>><<<<>>>><<<>>>><><<<<>>>><<>>><<<>>>><<>>>><<<>>>><<<>><<<>><<>><<>>>><>>><>>>><<<<>>><<<>><<>>>><>>>><<<><<><<<>><<<<>>>><<<><<<<>><<><<<>><<<<><>><<<<><<>>>><>>>><<><<<<>><<>><<<>>>><<>>>><<<<>>><<><<<>>>><<<<>>>><<<>>><>>>><<>>><<<<>>><<<<>>><<<<>>>><<>>>><<>>>><<<<>>>><<>>>><<>>><>><<<<>>><>>>><>><<>>><<<<>><<<>><<><<>>>><<<>><<>><>><<<>>>><<><<><<><<><<>>>><<<>><<>>>><>>><<<<><<><>>><<<><<<<>>>><<<>>><<<<>><>>>><>><<<>>><<<>>><<>><<<><<<<>>>><<<><<><<<>>><<<<>><>>><>>><<<<>>><><<<<>><<>>>><>><<>>><<><<>>><<<<>>>><>><<<><<<>>>><<<>>>><<<>><<<>>>><<><<<<><<<<>><<>>>><>>><>>><<<<>><>><>><<<>>><<>><><>>><<<<><<<<>>>><>>><<<><<><>>><<<<>>>><><>><<<><<<>>><>>><<>><<>>>><><>><>><<<>><<>>>><<<>><<<<>>>><<<>><<>>><<<<>>>><<<<><<<>><<<><>>>><<<<>>>><<<>><<<><<<>><>>>><>>><<><>><<>>>><<<<><><<>>><<<<>>><<<<>>><<>>><<>><<<<>>>><<<<>>>><><><<>><>>>><<>>><<<<>>>><<>><<<>><<<<>>><<>>><><<>><<>><<<>>>><>><<><<<<>><<<>><<<>>>><>><<<<>><><>>>><<<<>><<<>>><<<<>><<<>><><>>>><<>><>>>><<<<>>>><<<<>>><<<<><<<<><<>>><<<<>>><<<<>>><<<<>>>><>><<><<>><<>>>><<<<>>><<<<>><<<>><>>><<<<><<>>><<<>>>><<><<<>>>><<><>><<><><<<><<<<>>><>>><<>>>><<<<><<>><>>>><<>>>><<<<>>>><<<<>>><><<<<>>>><<>><<>><<<>>>><<>>><>>><>>>><<<<>>><<>>><><>>><<><>>>><<<<>>><<<>><<>><>>>><<><<<>>><<<>>>><<<>>>><<>>>><<<<>>>><<<<><<>>><<>>>><<>>>><<><<<>><<<<><<<<>><<>>>><<><<<<>>><<<><<<<>>><<<>>>><<>>><>><<>>><>>><<>><>>><>>>><<>><<<><<>>>><>>>><<<><>>>><<<<>>>><<<<><><>>><<<>>>><><>>>><<<>><>>><<<>>>><<><<><<<<>>>><>>><<<>><<<><>><<<><><<<>>>><<>><<><<<>>>><<<>>><<<>>>><<<>>>><<<><<>><<<>><><<<>>>><<<<>>><<>><>>>><>><<>>>><<<><<>><<<><<<>>>><<<<><<<<><><<<>>>><><<<<><<<>>>><<<<>><<>><><<<>>><>><<<<>><<<>>>><>>>><<<>>><<<<><<<>>><<>>>><<<<>><<<<>>><><<><<<<><<>><>>><>><>><<<<>>>><><<>>><<<<>>>><<<<>>>><<<>><><<<<>><>>>><<>>><<<<>>><<<>>>><<>>>><<<<><<<<>>><<<>>><<>>>><<<>>>><<<>>>><<<<><<<<>>><<>>>><>>><>><<>><<<>><<<>><>><<>><<><<<<>>>><<<>><<>><<<<>>><<<><><>><<<><<>><<<<>><<>>>><<><<<<>><<<<>>><<<<>>>><<<<>><<<<>><<<>>><>>>><<<>>><<<<>>>><><<<>>><<>>>><<<><<<>><<>>><>>><<>>><>>><<>>><<<><<<>>>><<>><<>><<>>><<>><<<>>>><<<>>>><<<>>><<<><><>>>><<<<><<<>>><<<><>><<<<>>>><<<>><<<<>>><<<>><><<<>><<<<>>><<<><<<>>>><><<<<>>><<<>><<<>><<<<>>><<<<>>><<<<>>>><<>>>><<<<>>><>><<<><<<><>><<<>>><<<>><<<<>><<<<><<><>>>><<><<<>>><<<<>>>><<>><>>>><<<<>>>><<><<<<>><<<<>>><<<>>>><>>>><>><<>><<<><><<<<>>>><><>>><<<>>><<<<>>><<<<>>>><<><<<>>><<>>>><<<><>><><<<<>>>><<<<>>>><<<>>><<<>>>><<>>>><<<>>>><<<>>><>><<<<>><><>><<<<>>>><>>>><<<>><<>>>><<><<<<>>><>>><<<>>>><<<<><<><>>><<<<><<<<>>><<>><><<<<>>>><<<>><>>>><>>>><<>><<<>>>><<<<>>><<<>>>><<<>>>><<>>><<<>>><<<<>><<<<>><>>><<>>><<>>>><<<<>>>><>>><>><<><>>><<<><<>>>><><<<>>><<<><<>>>><<<>>><><<<<>>><<><>>><<>>><>>>><><>><<>>><<>>>><<>>>><<<<>>><>><><<<<>>>><<<<>>><><<>><<<<>><<>><<<>>><<<<>>><<<>>><<<<>><><<<><<<>>>><>>>><<<<>><<<>>><<<><<><<<><>><><<<>>>><<<<>>><<<<>>><<<<><<<>>>><<<<>>>><<<>>><>><><<>><<<>><<><<<<>><<><<<>>><<>>><<>>>><<>>>><><>>>><<<>>>><<<<><<>><<<<>>>><<<>><<<<>><><>>><<<<>>>><<>>><<<>><<<>>><<>>>><<<><<<<>><<>><<<>><<>>>><<>><<<<>>><>>><>><>>>><<<>>><<>>><<<>><<>>><<<<><<<><<>>><<<<>>><>><><<<<><>><>>><>><<><<<<><<>>><>><>><<>>><<>>><>>><<<<>>><<<<>>><<><>><<<<>>><<<<><<>><<<><><>>><>>><<<<>>><<<><<<>><<><>>>><<>><<>><<<<><<<<>><<>>>><>><<><<<>>>><<<>>>><<><<<<>>>><<<>><<<>>>><<<><>>>><<<>><<>><<<>><<<<>>>><<<>>><<<>><<<>>>><<<>>><<>>><<<<>>>><>>>><<>>>><><<<<>><<><<><<>><>>><<>>>><<>>><<<<><<><>><<<<><>><<<<><<<<>><<>>><<<<><>>>><>><<<>>>><<<<>>>><<>><<<<>>><<>>>><<>>><<<<>><<<><<<<>><<<>>><<<>>>><<<>><>><<>>>><<><<><>><>>><>><<<>>><<>>><<<><<<<>>>><<<<>>><<<<>>><<>>><<<<>>>><<><><><>><<>>><<<>>><<<<><>>><>><>><<<<><<<<>><<<<>>><>>>><<<>>><<<>>>><<<>>>><<<<>>>><>>><<<<>>>><><<<><<>>><<<<>>><<>><<>><<<<>>>><<>><><<<<>><<<>>><<<>>><<<<>><<<>>>><<<<>>>><>>><>>>><<>>><<><<><<<>>><<<><<<><>>>><<<<><<>>><<<>><<<<><<<>>><><<<<>>>><<>>>><>>><>><>>>><<<<><<<><<<><<><<>>>><<>>><>>>><<<><<>>><<<>><<><<<><<<><<<>><>>><<<>><>>><<>><>><<<<>>>><<<<>>>><><<<<><<><<<<>>>><>>><<<>>>><>>>><<<>>><<<>>>><<<>>><<<>>>><>>><<<>><>>><<>>>><><<<>><<<><>>>><<<>>>><<<<><<<>>>><>>><<<>><<>><<<<>><<<><<>>><<<>>>><>>>><<<<>>><<<>><>>>><><<<<><<>>>><>>>><<<><<>>>><<<<>>><>>><>>>><><<>>>><<><><><<>><<>>><<<><<>>>><<><<>><<>>>><<<>>>><<><<><<>>><<<>><<<>>>><><<<>>>><<<<><>>>><<<>>><<>><>>>><<<<><<>>><<><>><'

In [138]:
def get_wind():
    index = 0
    while True:
        yield wind_input[index]
        index += 1
        if index == len(wind_input):
            index = 0

In [54]:
# helper function to print the chasm and pieces in a readable way
def print_chasm(c, ht=0, bot=0, piece=None):
    coords = set()
    if piece:
        coords = set(piece.get_coords())
        
    if ht == 0:
        ht = len(c)
    else:
        ht += 1
        
    for i, r in enumerate(reversed(c[bot:ht])):
        for j, c in enumerate(r):
            if(ht-i-1, j) in coords:
                print('@', end='')
            else:
                print(c, end='')
        if (ht-i-1) % 5 == 0:
            print(' ',abs(ht-i-1))
        else:
            print('')

In [145]:
def place_piece(p, h):
    coords = p.get_coords()
    for y, x in coords:
        chasm[y][x] = '#'
        if y > h:
            h = y
    return h

In [164]:
# code for part 1

# generate the chasm (tetris playing field)
check_num_blocks = 2022
chasm = [[] for _ in range(0, (((2022//5)+1)*13)+1)]  # there are 5 pieces repeating in order, so every 5 pcs is 13 tall
for i, _ in enumerate(chasm):
    chasm[i] = ['|'] + ['.' for _ in range(0,7)] + ['|']
chasm[0] = ['+'] + ['-' for _ in range(0,7)] + ['+']


highest_used = 0
wind = get_wind()


for count in range( 1, check_num_blocks+1 ):
    b = block(count, highest_used)
    moving = True
    while moving:
        b.move(chasm, next(wind))
        moving = b.move(chasm, 'down')
    highest_used = place_piece(b, highest_used)

print(highest_used)

3215
